# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
#Align text to the left
def left_align(df):
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'}).set_table_styles(
        [ dict(selector='th', props=[('text-align', 'left')] ) ])
    
    return left_aligned_df

In [3]:
#Import csv from WeatherPy and create a Data Frame
weatherpy_df = pd.read_csv('../output_data/cities.csv')
left_align(weatherpy_df.head(20))

,City,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Chimoio,-19.116400,33.483300,68.700000,94,54,3.180000,MZ,1612816032
1,San jeronimo,-11.947200,-75.282900,64.890000,45,76,3.330000,PE,1612816032
2,Nishihara,26.184200,127.755800,60.800000,58,1,8.050000,JP,1612816032
3,Vaini,-21.200000,-175.200000,78.800000,94,90,10.360000,TO,1612816032
4,Pevek,69.700800,170.313300,-11.920000,83,100,1.700000,RU,1612816032
5,Khatanga,71.966700,102.500000,-19.210000,82,0,14.050000,RU,1612816033
6,Busselton,-33.650000,115.333300,55.990000,98,79,8.520000,AU,1612815770
7,Victoria,22.285500,114.157700,66.000000,78,99,1.990000,HK,1612815844
8,Rikitea,-23.120300,-134.969200,76.840000,73,20,8.340000,PF,1612816033
9,Albany,42.600100,-73.966200,27.000000,39,87,4.230000,US,1612816033


### Humidity Heatmap

In [10]:
heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 5

NameError: name 'heatmap_layer' is not defined

In [26]:
#Set locations based on latitude and Longitude
locations = weatherpy_df[['Latitude', 'Longitude']]

figure_layout = {
    'width': '600px',
    'height': '600px',

}

heat_layer.max_intensity = 100
heat_layer.point_radius = 1

#Adding Heatmap layer based on Humidity percentages
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights = weatherpy_df['Humidity (%)'],
                                 dissipating = False, max_intensity=10, point_radius=1)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='600px', width='600px'))

### Create new DataFrame fitting weather criteria

In [5]:
#Create New Data Frame based on my ideal weather conditions for vacation
ideal_weather_df = weatherpy_df[(weatherpy_df['Max Temp (F)'] <= 60) &
                                (weatherpy_df['Max Temp (F)'] <= 70) & 
                                (weatherpy_df['Wind Speed (mph)'] <= 10) & 
                                (weatherpy_df['Cloudiness (%)'] <= 25)]
#Dropped Null values
ideal_weather_df = ideal_weather_df.dropna()
left_align(ideal_weather_df.head())

,City,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
13,Lasa,46.616600,10.700200,42.800000,79,0,3.600000,IT,1612816033
18,Norman wells,65.282000,-126.832900,-29.200000,75,1,9.220000,CA,1612816034
20,Upernavik,72.786800,-56.154900,15.980000,77,0,2.620000,GL,1612816034
46,Saskylakh,71.916700,114.083300,-31.090000,83,25,3.580000,RU,1612816037
57,Seydi,39.481600,62.913700,44.560000,73,0,8.250000,TM,1612816038


### Call API and collect data

In [6]:
#Empty lists to store info collected
hotel_name = []
hotel_lat = []
hotel_lng = []
hotel_city = []
hotel_country = []

for index, row in ideal_weather_df.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    parameters = {
                 'location':f"{lat}, {lng}",
                 'radius': 5000,
                 'type':'hotel',
                 'key': g_key
                 }
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    response = requests.get(base_url, params=parameters)
    hotel_data = response.json()    

    try:
        hotel_name.append(hotel_data['results'][0]['name'])
        hotel_lat.append(hotel_data['results'][0]['geometry']['location']['lat'])
        hotel_lng.append(hotel_data['results'][0]['geometry']['location']['lng'])
        hotel_city.append(row['City'])
        hotel_country.append(row['Country'])
        
    except:
        print('No hotel found within 5000 meters radius')

### Create new Data Frame with Data collected

In [7]:
hotel_df = pd.DataFrame({'Hotel Name': hotel_name,
                        'City': hotel_city,
                        'Country': hotel_country,
                        'Latitude': hotel_lat,
                        'Longitude': hotel_lng
                        })
hotel_df['City'] = hotel_df['City'].str.capitalize()
left_align(hotel_df.head(20))

,Hotel Name,City,Country,Latitude,Longitude
0,Schlanders,Lasa,IT,46.627677,10.773689
1,Norman Wells,Norman wells,CA,65.281494,-126.828652
2,Upernavik,Upernavik,GL,72.786288,-56.137553
3,Saskylakh,Saskylakh,RU,71.954308,114.119835
4,Seydi,Seydi,TM,39.428637,62.861760
5,Fairbanks,Fairbanks,US,64.837778,-147.716389
6,Pangnirtung,Pangnirtung,CA,66.146558,-65.701218
7,Kalifornsky,Kenai,US,60.418333,-151.290000
8,Tuktoyaktuk,Tuktoyaktuk,CA,69.445358,-133.034181
9,Saint Petersburg,Nikolskoye,RU,59.931058,30.360910


### Create Info Box

In [8]:
# Creating Info Box Template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
coordinates = hotel_df[['Latitude', 'Longitude']]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(coordinates, info_box_content=hotel_info)

# Display figure
fig2 = gmaps.figure()
fig2.add_layer(markers)
fig2

Figure(layout=FigureLayout(height='420px'))